In [20]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler


# Phase 1: Data Loading & Initial Inspection

df = pd.read_csv("C:\\Users\\Lenovo\\Downloads\\orders1.csv")

#Display the first few rows to understand the structure
print("---- First 5 Rows ----")
print(df.head())

#Display data types of each column
print("\n---- Data Types ----")
print(df.dtypes)

#Print dataset dimensions
print(f"\nDataset Dimensions: {df.shape[0]} rows, {df.shape[1]} columns")

#Check for missing values in each column
print("\n---- Missing Values ----")
print(df.isnull().sum())

#Basic descriptive statistics for numerical columns
print("\n---- Summary Statistics ----")
print(df.describe())

#Check for duplicate rows
duplicates = df.duplicated().sum()
print(f"\nDuplicate Rows: {duplicates}")

---- First 5 Rows ----
   Row ID        Order ID  Order Date   Ship Date       Ship Mode Customer ID  \
0       1  CA-2016-152156  08-11-2016  11-11-2016    Second Class    CG-12520   
1       2  CA-2016-152156  08-11-2016  11-11-2016    Second Class    CG-12520   
2       3  CA-2016-138688  12-06-2016  16-06-2016    Second Class    DV-13045   
3       4  US-2015-108966  11-10-2015  18-10-2015  Standard Class    SO-20335   
4       5  US-2015-108966  11-10-2015  18-10-2015  Standard Class    SO-20335   

     Customer Name    Segment        Country             City  ...  \
0      Claire Gute   Consumer  United States        Henderson  ...   
1      Claire Gute   Consumer  United States        Henderson  ...   
2  Darrin Van Huff  Corporate  United States      Los Angeles  ...   
3   Sean O'Donnell   Consumer  United States  Fort Lauderdale  ...   
4   Sean O'Donnell   Consumer  United States  Fort Lauderdale  ...   

  Postal Code  Region       Product ID         Category Sub-Category 

In [22]:
# Phase 2: Data Cleaning


#Convert 'Order Date' and 'Ship Date' to datetime using the correct format
df['Order Date'] = pd.to_datetime(df['Order Date'], format='%d-%m-%Y', errors='coerce')
df['Ship Date'] = pd.to_datetime(df['Ship Date'], format='%d-%m-%Y', errors='coerce')

#Verify that date conversion did not produce NaT values (if there are any errors in this, they will appear in the output:)
if df['Order Date'].isnull().any() or df['Ship Date'].isnull().any():
    print("\nDate conversion errors detected. Please check the date format in your CSV.")
else:
    print("\nDates converted successfully.")

#Fill missing values
#FOR EXAMPLE: For Postal Code, fill missing values with the mode (most frequent value)
if df['Postal Code'].isnull().sum() > 0:
    df['Postal Code'].fillna(df['Postal Code'].mode()[0], inplace=True)

#Remove duplicate rows
df.drop_duplicates(inplace=True)

#Standardize text columns for consistency (it converts to title case and strip extra spaces)
for col in ['Customer Name', 'City', 'State', 'Country']:
    if col in df.columns:
        df[col] = df[col].str.title().str.strip()

#Handle outliers in numerical columns (Sales and Profit) using winsorization(capping)
def cap_outliers(series, lower_quantile=0.05, upper_quantile=0.95):
    lower = series.quantile(lower_quantile)
    upper = series.quantile(upper_quantile)
    return np.clip(series, lower, upper)

df['Sales'] = cap_outliers(df['Sales'])
df['Profit'] = cap_outliers(df['Profit'])


Dates converted successfully.


In [24]:
# Phase 3: Data Transformation
# ============================

# Feature Engineering: Create a new column for 'Processing Time' (in days)
df['Processing Time'] = (df['Ship Date'] - df['Order Date']).dt.days

# Feature Engineering: Create 'Revenue Category' based on the Sales column
df['Revenue Category'] = pd.cut(df['Sales'],
                                bins=[0, 100, 500, 1000, 5000, np.inf],
                                labels=['Very Low', 'Low', 'Medium', 'High', 'Very High'])

# Data Aggregation: Calculate total sales per customer
customer_sales = df.groupby('Customer ID')['Sales'].sum().reset_index()
customer_sales.rename(columns={'Sales': 'Total Sales'}, inplace=True)
print("\n---- Total Sales per Customer (Sample) ----")
print(customer_sales.head())

# Data Aggregation: Calculate average discount by product category
category_discount = df.groupby('Category')['Discount'].mean().reset_index()
category_discount.rename(columns={'Discount': 'Average Discount'}, inplace=True)
print("\n---- Average Discount by Category (Sample) ----")
print(category_discount.head())

# Data Standardization/Normalization: Normalize the 'Profit' column (keeping Sales unchanged for Revenue Category)
scaler = MinMaxScaler()
df['Normalized Profit'] = scaler.fit_transform(df[['Profit']])
print("\n---- Normalized Profit (First 5 Rows) ----")
print(df[['Profit', 'Normalized Profit']].head())


---- Total Sales per Customer (Sample) ----
  Customer ID  Total Sales
0    AA-10315  2593.148245
1    AA-10375  1056.390000
2    AA-10480  1790.512000
3    AA-10645  4397.838490
4    AB-10015   886.156000

---- Average Discount by Category (Sample) ----
          Category  Average Discount
0        Furniture          0.173923
1  Office Supplies          0.157285
2       Technology          0.132323

---- Normalized Profit (First 5 Rows) ----
      Profit  Normalized Profit
0   41.91360           0.428641
1  168.47040           1.000000
2    6.87140           0.270438
3  -53.03092           0.000000
4    2.51640           0.250776


In [26]:
#Phase 4: Reporting & Documentation


#create the directory first
import os
os.makedirs("/mnt/data", exist_ok=True)
#save the cleaned and transformed dataset to a new CSV file
output_path = "/mnt/data/cleaned_orders.csv"
df.to_csv(output_path, index=False)
print(f"\nCleaned dataset saved to: {output_path}")

#Report Summary:
# 1.Loaded the dataset and inspected its structure.
# 2.Converted date columns and handled missing values.
# 3.Removed duplicates and standardized text fields.
# 4.Handled outliers in Sales and Profit using winsorization.
# 5.Engineered features (Processing Time, Revenue Category) and aggregated data for insights.
# 6.Normalized the Profit column for additional analysis.



Cleaned dataset saved to: /mnt/data/cleaned_orders.csv
